In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# pandas 설정: 각 column 별 (혹은 한 cell에서) 출력되는 글자수 제한을 없애기
pd.set_option('display.max_colwidth', -1)
pd.__version__

'1.1.4'

In [3]:
df = pd.read_csv("data/koelectra_result/KoElectra_small_class_epoch50_prob.csv", encoding="utf-8")
df = df[["qplay_question_id","predict_category","predict_middle_category","predict_probability","true_category","true_middle_category"]]
df

,qplay_question_id,predict_category,predict_middle_category,predict_probability,true_category,true_middle_category
0,8548,HSU1-01-12,HSU1-01,"{'HSU1-01-12': -11.376259, 'HSU1-01-11': -13.564799, 'HSU1-01-10': -17.119465, 'H1S1-04-08': -18.028997, 'H1S2-05-11': -18.051655, 'H1S2-04-19': -18.181334, 'H1S2-05-03': -18.232037, 'H1S1-01-03': -18.64543, 'HSU1-03-01': -19.009594, 'H1S2-04-02': -19.121258, 'H1S2-05-05': -19.32013}",HSU1-01-12,HSU1-01
1,9148,HSTA-07-07,HSTA-07,"{'HSTA-07-07': -8.819035, 'HSTA-07-09': -9.240341, 'HSTA-07-10': -10.081232, 'HSTA-07-11': -13.904758, 'HSTA-07-08': -13.9349785, 'HSTA-07-05': -15.334319, 'HSTA-07-13': -15.531416, 'H1S1-02-12': -15.536122, 'HSTA-06-10': -16.00332, 'HSTA-02-07': -16.17024, 'HSTA-06-04': -16.40165}",HSTA-07-08,HSTA-07
2,17608,H1S2-04-20,H1S2-04,"{'H1S2-04-20': -11.012324, 'H1S2-04-19': -16.903236, 'H1S2-04-14': -17.586264, 'H1S2-04-10': -18.533783, 'H1S2-05-25': -19.170769, 'H1S2-04-16': -19.198435, 'H1S1-12-12': -19.301615, 'H1S2-04-15': -19.351856, 'H1S2-04-12': -19.612349, 'H1S1-02-07': -19.881647, 'H1S2-04-11': -20.07652}",H1S2-04-19,H1S2-04
3,7788,HSU1-01-04,HSU1-01,"{'HSU1-01-04': -17.317554, 'H1S1-09-03': -19.275858, 'HSU1-03-05': -19.289806, 'HSU1-01-02': -20.436504, 'HSU1-01-07': -21.663912, 'H1S1-10-02': -22.735518, 'HSTA-05-09': -22.835047, 'H1S1-09-11': -23.01901, 'H1S1-11-03': -23.068302, 'H1S2-07-15': -23.283081, 'H1S1-10-04': -23.606974}",HSU1-03-05,HSU1-03
4,25275,H1S2-04-20,H1S2-04,"{'H1S2-04-20': -11.5843115, 'H1S2-04-19': -12.056982, 'H1S2-04-10': -13.817684, 'H1S2-04-12': -14.2238035, 'H1S2-04-11': -15.567081, 'H1S2-06-14': -16.433693, 'H1S2-04-14': -17.520672, 'H1S2-05-25': -18.32765, 'H1S2-04-16': -19.267708, 'H1S2-04-13': -19.285646, 'H1S1-04-08': -19.891586}",H1S2-04-19,H1S2-04
...,...,...,...,...,...,...
1065,22282,HSU1-11-12,HSU1-11,"{'HSU1-11-12': -8.492875, 'H1S2-03-18': -8.78329, 'HSU1-11-11': -13.091569, 'H1S2-03-17': -14.731253, 'HSU1-02-09': -15.079092, 'H1S2-03-19': -15.252344, 'HSU1-11-13': -15.266082, 'H1S1-04-08': -15.358758, 'HSU1-11-06': -15.568064, 'H1S2-05-04': -16.099043, 'HSU1-10-06': -16.209307}",H1S2-03-18,H1S2-03
1066,4693,H1S1-11-08,H1S1-11,"{'H1S1-11-08': -10.096279, 'H1S1-11-09': -14.659082, 'H1S1-11-11': -17.246206, 'H1S1-11-10': -17.34268, 'H1S1-12-04': -18.34105, 'H1S1-09-05': -18.367018, 'H1S1-10-10': -18.452984, 'H1S1-02-06': -19.113699, 'H1S1-11-14': -19.15764, 'H1S1-01-04': -19.469738, 'H1S1-11-06': -19.644596}",H1S1-11-08,H1S1-11
1067,24849,H1S2-07-11,H1S2-07,"{'H1S2-07-11': -1.884048, 'H1S2-07-20': -2.383606, 'H1S2-07-10': -4.3870173, 'HSTA-03-03': -6.2709527, 'H1S2-07-14': -7.863308, 'HSTA-01-01': -8.188512, 'H1S2-07-07': -8.323971, 'HSTA-04-03': -8.563311, 'H1S2-07-12': -8.762451, 'HSTA-01-09': -8.838194, 'HSTA-01-04': -9.000531}",H1S2-07-11,H1S2-07
1068,3391,H1S1-11-17,H1S1-11,"{'H1S1-11-17': -13.7485, 'H1S1-11-16': -14.080412, 'H1S1-11-12': -14.629485, 'H1S1-11-05': -15.886945, 'H1S1-12-04': -16.623337, 'H1S1-11-18': -16.796282, 'H1S1-10-12': -16.945255, 'H1S1-12-07': -17.744617, 'H1S1-12-06': -17.863874, 'H1S1-10-10': -17.97471, 'H1S1-12-10': -17.990192}",H1S1-11-16,H1S1-11


In [4]:
# convert value's string type in column to dictionary
import ast
df['predict_probability'] = df['predict_probability'].apply(lambda x: ast.literal_eval(x))

In [5]:
import random
index_no = random.randint(0, len(df))
dict_sample = df["predict_probability"].iloc[index_no]
print(dict_sample)

{'H1S2-05-17': -10.327636, 'H1S2-06-08': -17.325796, 'H1S2-05-19': -17.458927, 'H1S2-06-01': -17.709557, 'H1S1-04-04': -17.88108, 'H1S1-06-06': -18.194096, 'H1S2-05-13': -18.474024, 'H1S2-06-11': -18.555866, 'H1S1-06-05': -18.659866, 'HSU1-03-07': -18.713749, 'H1S1-05-05': -18.79045}


In [6]:
list_sample_ch = dict_sample.keys()
list_sample_middle_ch = [w[:-3] for w in list_sample_ch]
print(list_sample_middle_ch)

['H1S2-05', 'H1S2-06', 'H1S2-05', 'H1S2-06', 'H1S1-04', 'H1S1-06', 'H1S2-05', 'H1S2-06', 'H1S1-06', 'HSU1-03', 'H1S1-05']


In [7]:
def most_frequent(List): 
    return max(set(List), key = List.count) 

def get_democracy(dict_sample):
    list_sample_ch = dict_sample.keys()
    list_sample_middle_ch = [w[:-3] for w in list_sample_ch]
    str_frequent = most_frequent(list_sample_middle_ch)
    return str_frequent

get_democracy(dict_sample)

'H1S2-05'

In [8]:
import numpy as np

def get_variance(dict_sample):
    list_sample_loss = list(dict_sample.values())
    variance = np.var(list_sample_loss)
    return variance

get_variance(dict_sample)

5.33435032750443

In [9]:
def get_mean(dict_sample):
    list_sample_loss = list(dict_sample.values())
    mean = np.mean(list_sample_loss)
    return mean
get_mean(dict_sample)

-17.462822454545456

In [10]:
import numpy as np

def get_maximum(sample_dict):
    list_sample_loss = list(sample_dict.values())
    maximum = np.max(list_sample_loss)
    return maximum

In [11]:
df["frequent_prob"] = df['predict_probability'].apply(lambda x: get_democracy(x))
df["mean_prob"] = df['predict_probability'].apply(lambda x: get_mean(x))
df["var_prob"] = df['predict_probability'].apply(lambda x: get_variance(x))
df["max_prob"] = df['predict_probability'].apply(lambda x: get_maximum(x))
print(df.shape)
df

(1070, 10)


,qplay_question_id,predict_category,predict_middle_category,predict_probability,true_category,true_middle_category,frequent_prob,mean_prob,var_prob,max_prob
0,8548,HSU1-01-12,HSU1-01,"{'HSU1-01-12': -11.376259, 'HSU1-01-11': -13.564799, 'HSU1-01-10': -17.119465, 'H1S1-04-08': -18.028997, 'H1S2-05-11': -18.051655, 'H1S2-04-19': -18.181334, 'H1S2-05-03': -18.232037, 'H1S1-01-03': -18.64543, 'HSU1-03-01': -19.009594, 'H1S2-04-02': -19.121258, 'H1S2-05-05': -19.32013}",HSU1-01-12,HSU1-01,HSU1-01,-17.331905,5.812408,-11.376259
1,9148,HSTA-07-07,HSTA-07,"{'HSTA-07-07': -8.819035, 'HSTA-07-09': -9.240341, 'HSTA-07-10': -10.081232, 'HSTA-07-11': -13.904758, 'HSTA-07-08': -13.9349785, 'HSTA-07-05': -15.334319, 'HSTA-07-13': -15.531416, 'H1S1-02-12': -15.536122, 'HSTA-06-10': -16.00332, 'HSTA-02-07': -16.17024, 'HSTA-06-04': -16.40165}",HSTA-07-08,HSTA-07,HSTA-07,-13.723401,7.727426,-8.819035
2,17608,H1S2-04-20,H1S2-04,"{'H1S2-04-20': -11.012324, 'H1S2-04-19': -16.903236, 'H1S2-04-14': -17.586264, 'H1S2-04-10': -18.533783, 'H1S2-05-25': -19.170769, 'H1S2-04-16': -19.198435, 'H1S1-12-12': -19.301615, 'H1S2-04-15': -19.351856, 'H1S2-04-12': -19.612349, 'H1S1-02-07': -19.881647, 'H1S2-04-11': -20.07652}",H1S2-04-19,H1S2-04,H1S2-04,-18.238982,6.058112,-11.012324
3,7788,HSU1-01-04,HSU1-01,"{'HSU1-01-04': -17.317554, 'H1S1-09-03': -19.275858, 'HSU1-03-05': -19.289806, 'HSU1-01-02': -20.436504, 'HSU1-01-07': -21.663912, 'H1S1-10-02': -22.735518, 'HSTA-05-09': -22.835047, 'H1S1-09-11': -23.01901, 'H1S1-11-03': -23.068302, 'H1S2-07-15': -23.283081, 'H1S1-10-04': -23.606974}",HSU1-03-05,HSU1-03,HSU1-01,-21.502870,4.016087,-17.317554
4,25275,H1S2-04-20,H1S2-04,"{'H1S2-04-20': -11.5843115, 'H1S2-04-19': -12.056982, 'H1S2-04-10': -13.817684, 'H1S2-04-12': -14.2238035, 'H1S2-04-11': -15.567081, 'H1S2-06-14': -16.433693, 'H1S2-04-14': -17.520672, 'H1S2-05-25': -18.32765, 'H1S2-04-16': -19.267708, 'H1S2-04-13': -19.285646, 'H1S1-04-08': -19.891586}",H1S2-04-19,H1S2-04,H1S2-04,-16.179711,7.939232,-11.584312
...,...,...,...,...,...,...,...,...,...,...
1065,22282,HSU1-11-12,HSU1-11,"{'HSU1-11-12': -8.492875, 'H1S2-03-18': -8.78329, 'HSU1-11-11': -13.091569, 'H1S2-03-17': -14.731253, 'HSU1-02-09': -15.079092, 'H1S2-03-19': -15.252344, 'HSU1-11-13': -15.266082, 'H1S1-04-08': -15.358758, 'HSU1-11-06': -15.568064, 'H1S2-05-04': -16.099043, 'HSU1-10-06': -16.209307}",H1S2-03-18,H1S2-03,HSU1-11,-13.993789,6.984534,-8.492875
1066,4693,H1S1-11-08,H1S1-11,"{'H1S1-11-08': -10.096279, 'H1S1-11-09': -14.659082, 'H1S1-11-11': -17.246206, 'H1S1-11-10': -17.34268, 'H1S1-12-04': -18.34105, 'H1S1-09-05': -18.367018, 'H1S1-10-10': -18.452984, 'H1S1-02-06': -19.113699, 'H1S1-11-14': -19.15764, 'H1S1-01-04': -19.469738, 'H1S1-11-06': -19.644596}",H1S1-11-08,H1S1-11,H1S1-11,-17.444634,7.194505,-10.096279
1067,24849,H1S2-07-11,H1S2-07,"{'H1S2-07-11': -1.884048, 'H1S2-07-20': -2.383606, 'H1S2-07-10': -4.3870173, 'HSTA-03-03': -6.2709527, 'H1S2-07-14': -7.863308, 'HSTA-01-01': -8.188512, 'H1S2-07-07': -8.323971, 'HSTA-04-03': -8.563311, 'H1S2-07-12': -8.762451, 'HSTA-01-09': -8.838194, 'HSTA-01-04': -9.000531}",H1S2-07-11,H1S2-07,H1S2-07,-6.769627,6.463771,-1.884048
1068,3391,H1S1-11-17,H1S1-11,"{'H1S1-11-17': -13.7485, 'H1S1-11-16': -14.080412, 'H1S1-11-12': -14.629485, 'H1S1-11-05': -15.886945, 'H1S1-12-04': -16.623337, 'H1S1-11-18': -16.796282, 'H1S1-10-12': -16.945255, 'H1S1-12-07': -17.744617, 'H1S1-12-06': -17.863874, 'H1S1-10-10': -17.97471, 'H1S1-12-10': -17.990192}",H1S1-11-16,H1S1-11,H1S1-11,-16.389419,2.297384,-13.748500


In [12]:
df = df[["qplay_question_id","true_middle_category","predict_middle_category", "frequent_prob","mean_prob", "var_prob", "max_prob"]]
df.sample(5)

,qplay_question_id,true_middle_category,predict_middle_category,frequent_prob,mean_prob,var_prob,max_prob
926,3536,H1S1-05,H1S1-05,HSU1-11,-19.632886,5.122058,-12.849252
708,3439,H1S1-07,H1S1-07,H1S1-07,-13.151000,4.133703,-9.513939
279,4555,H1S1-03,H1S1-02,H1S1-03,-16.726286,5.973888,-10.138098
585,25205,H1S2-04,H1S2-04,H1S2-04,-15.022659,7.969217,-7.638349
873,21055,H1S2-06,H1S2-06,H1S2-05,-16.999241,7.579874,-8.512825


In [13]:
index_diff = df["true_middle_category"].compare(df["predict_middle_category"]).index
df_maximum = df.loc[index_diff]
df_maximum

,qplay_question_id,true_middle_category,predict_middle_category,frequent_prob,mean_prob,var_prob,max_prob
3,7788,HSU1-03,HSU1-01,HSU1-01,-21.502870,4.016087,-17.317554
16,8862,HSU1-07,H1S2-05,HSU1-07,-20.835493,1.890619,-17.767733
36,4033,H1S1-03,H1S1-01,H1S1-03,-14.293819,4.503212,-8.133976
44,20361,H1S2-01,H1S2-02,HSTA-01,-21.091686,1.206112,-18.217182
59,8563,HSU1-01,HSU1-02,HSU1-02,-23.591778,4.416673,-17.689194
...,...,...,...,...,...,...,...
1046,8099,HSU1-02,HSU1-01,HSU1-01,-20.004243,2.230362,-16.981804
1047,24379,H1S2-04,H1S2-06,H1S2-03,-23.561338,0.405294,-22.196596
1056,8322,HSU1-02,HSU1-09,H1S2-07,-12.255645,1.007911,-10.891567
1058,5035,H1S1-03,H1S2-05,H1S1-03,-23.016523,0.646225,-21.622854


In [22]:
df_maximum.sort_values(by="var_prob", ascending=False)[:15]

,qplay_question_id,true_middle_category,predict_middle_category,frequent_prob,mean_prob,var_prob,max_prob
685,9076,HSTA-01,H1S2-07,H1S2-07,-12.231633,12.802036,-2.052586
672,9612,HSTA-01,H1S2-07,H1S2-04,-14.505717,10.737273,-9.487900
325,24243,H1S2-07,HSTA-01,H1S2-07,-11.918290,10.517630,-4.653641
386,24347,H1S2-04,H1S2-07,H1S2-04,-14.648479,9.397514,-6.732120
83,31110,H1S2-07,H1S2-04,H1S2-04,-13.547303,9.028274,-5.242729
397,20018,H1S2-02,H1S2-07,H1S2-07,-13.858449,8.939008,-5.802656
146,24492,H1S2-05,H1S2-06,H1S2-05,-18.032093,8.399956,-9.503048
959,9558,HSTA-02,H1S2-07,H1S2-07,-17.118172,7.814893,-10.809184
446,9579,HSTA-02,H1S2-07,H1S2-04,-16.414425,7.701511,-8.639976
82,9636,HSTA-02,H1S2-07,H1S2-07,-17.460753,7.691984,-9.832176


In [23]:
df_maximum.sort_values(by="max_prob", ascending=False)[:20]

,qplay_question_id,true_middle_category,predict_middle_category,frequent_prob,mean_prob,var_prob,max_prob
685,9076,HSTA-01,H1S2-07,H1S2-07,-12.231633,12.802036,-2.052586
84,24579,H1S2-07,HSTA-01,HSTA-01,-8.593002,5.662738,-2.188203
704,24509,H1S2-06,H1S2-04,H1S2-04,-11.099396,5.918564,-3.543010
640,24563,H1S2-05,H1S2-06,H1S2-06,-12.299901,7.344244,-3.947406
788,9028,HSTA-01,H1S2-07,H1S2-07,-9.924389,6.571688,-4.308950
611,9211,HSTA-06,HSTA-04,HSTA-04,-9.274288,5.778295,-4.430949
325,24243,H1S2-07,HSTA-01,H1S2-07,-11.918290,10.517630,-4.653641
83,31110,H1S2-07,H1S2-04,H1S2-04,-13.547303,9.028274,-5.242729
65,9227,HSTA-07,HSTA-06,HSTA-06,-8.076564,1.614032,-5.371318
397,20018,H1S2-02,H1S2-07,H1S2-07,-13.858449,8.939008,-5.802656


In [15]:
index_diff = df["true_middle_category"].compare(df["frequent_prob"]).index
df_freq = df.loc[index_diff]
df_freq[:5]

,qplay_question_id,true_middle_category,predict_middle_category,frequent_prob,mean_prob,var_prob,max_prob
3,7788,HSU1-03,HSU1-01,HSU1-01,-21.502870,4.016087,-17.317554
10,8108,HSU1-05,HSU1-05,HSU1-06,-19.017443,5.409780,-12.095246
11,5045,H1S1-01,H1S1-01,HSU1-03,-11.361557,3.952362,-5.782053
12,5047,H1S1-01,H1S1-01,H1S1-03,-11.649529,4.640895,-5.952143
13,8241,HSU1-05,HSU1-05,HSU1-06,-19.686656,5.357581,-12.735401


In [25]:
df_freq.sort_values(by="var_prob", ascending=False)[:20]

,qplay_question_id,true_middle_category,predict_middle_category,frequent_prob,mean_prob,var_prob,max_prob
685,9076,HSTA-01,H1S2-07,H1S2-07,-12.231633,12.802036,-2.052586
672,9612,HSTA-01,H1S2-07,H1S2-04,-14.505717,10.737273,-9.487900
70,9121,HSTA-01,HSTA-01,H1S2-07,-11.978897,9.948903,-4.192392
270,25228,H1S2-07,H1S2-07,H1S2-03,-16.668663,9.331522,-9.266034
579,7575,HSTA-02,HSTA-02,HSU1-01,-16.787311,9.328562,-8.477284
176,25128,H1S2-07,H1S2-07,H1S2-04,-14.891567,9.285581,-7.243638
364,24691,H1S2-06,H1S2-06,HSU1-10,-19.481431,9.276710,-13.851152
559,25173,H1S2-07,H1S2-07,H1S2-03,-16.090738,9.037000,-7.404475
83,31110,H1S2-07,H1S2-04,H1S2-04,-13.547303,9.028274,-5.242729
397,20018,H1S2-02,H1S2-07,H1S2-07,-13.858449,8.939008,-5.802656


In [17]:
def get_mean_across_series():
    return